In [1]:
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import numpy as np
import pandas as pd 
from Bio import SeqIO
from collections import Counter, defaultdict, OrderedDict
import nltk
from nltk.util import ngrams
import re
from sklearn.linear_model import Lasso, LinearRegression, SGDClassifier, LogisticRegression
import itertools 
import operator
import func_tools 
from sklearn.model_selection import cross_val_score
import pickle
import seaborn as sb
from sklearn.preprocessing import LabelEncoder, StandardScaler, scale
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.cm as cm 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import f_regression, SelectKBest
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import chi2_contingency
import scipy.stats as scs
from itertools import combinations
%matplotlib inline 

/Users/Macbook/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [15]:
age3_df = pd.read_csv('data/age3/age3_df.csv', index_col='Unnamed: 0')
flu_df = pd.read_csv('data/flu/flu_df.csv', index_col='Unnamed: 0')

In [31]:
cls = LogisticRegression(penalty='l1')
clf = GridSearchCV(estimator=cls, param_grid={'C': np.linspace(1e-5, 1, 10)}, scoring='roc_auc')

clf.fit(scale(flu_df[['size', 'value1']]), flu_df.quality_imp)

print "age_df roc_auc: ", roc_auc_score(age3_df.quality_imp, clf.best_estimator_.predict_proba(scale(age3_df[['size', 'value1']]))[:,1])

age_df roc_auc:  0.962428492819


In [33]:
cls = LogisticRegression(penalty='l1')
clf = GridSearchCV(estimator=cls, param_grid={'C': np.linspace(1e-5, 1, 10)}, scoring='roc_auc')

clf.fit(np.hstack([flu_df[['second_vote_std', 'size', 'CDR_sum']],flu_df[['pos_ex_value_sum','pos1_in_CDR3','pos2_in_CDR3','pos3_in_CDR3']]]), flu_df.quality_imp)

print "age_df roc_auc: ", roc_auc_score(age3_df.quality_imp, clf.best_estimator_.predict_proba(age3_df[['second_vote_std', 'size', 'CDR_sum', 'pos_ex_value_sum','pos1_in_CDR3','pos2_in_CDR3','pos3_in_CDR3']])[:,1])

age_df roc_auc:  0.942997374223


In [30]:
cls = RandomForestClassifier(n_estimators=1000)
clf = GridSearchCV(estimator=cls, param_grid={'max_depth': [10]}, scoring='roc_auc')

clf.fit(flu_df[['value1', 'size']], flu_df.quality_imp)

print "age_df roc_auc: ", roc_auc_score(age3_df.quality_imp, clf.best_estimator_.predict_proba(age3_df[['size','value1']])[:,1])

age_df roc_auc:  0.5


In [29]:
cls = RandomForestClassifier(n_estimators=1000)
clf = GridSearchCV(estimator=cls, param_grid={'max_depth': [10]}, scoring='roc_auc')

clf.fit(flu_df[['second_vote_std', 'size', 'CDR_sum', 'pos_ex_value_sum','pos1_in_CDR3','pos2_in_CDR3','pos3_in_CDR3']], flu_df.quality_imp)

print "age_df roc_auc: ", roc_auc_score(age3_df.quality_imp, clf.best_estimator_.predict_proba(age3_df[['second_vote_std', 'size', 'CDR_sum', 'pos_ex_value_sum','pos1_in_CDR3','pos2_in_CDR3','pos3_in_CDR3']])[:,1])

 age_df roc_auc:  0.96899847476


In [41]:
cls = LogisticRegression(penalty='l1')
clf = GridSearchCV(estimator=cls, param_grid={'C': np.linspace(1e-5, 1, 10)}, scoring='roc_auc')

X_flu_train, X_flu_test, y_flu_train, y_flu_test = train_test_split(pd.DataFrame(scale(flu_df[['value1', 'size']]), index=flu_df.index), flu_df.quality_imp, 
                                                                    test_size=0.2, stratify = flu_df.quality_imp)
X_age3_train, X_age3_test, y_age3_train, y_age3_test = train_test_split(pd.DataFrame(scale(age3_df[['value1', 'size']]), index=age3_df.index), age3_df.quality_imp, 
                                                                    test_size=0.2, stratify = age3_df.quality_imp)
X_train = pd.concat([X_flu_train, X_age3_train])
X_test = pd.concat([X_flu_test, X_age3_test])
y_train = pd.concat([y_flu_train, y_age3_train])
y_test = pd.concat([y_flu_test, y_age3_test])

clf.fit(X_train, y_train)

print "age_df roc_auc: ", roc_auc_score(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])

age_df roc_auc:  0.960012247397


In [47]:
cls = LogisticRegression(penalty='l1')
clf = GridSearchCV(estimator=cls, param_grid={'C': np.linspace(1e-5, 1, 10)}, scoring='roc_auc')

X_flu_train, X_flu_test, y_flu_train, y_flu_test = train_test_split(pd.DataFrame(np.hstack([flu_df[['second_vote_std', 'size', 
                                                                    'CDR_sum']],flu_df[['pos_ex_value_sum','pos1_in_CDR3',
                                                                    'pos2_in_CDR3','pos3_in_CDR3']]]), index=flu_df.index), 
                                                                    flu_df.quality_imp, 
                                                                    test_size=0.2, stratify = flu_df.quality_imp)

X_age3_train, X_age3_test, y_age3_train, y_age3_test = train_test_split(pd.DataFrame(np.hstack([age3_df[['second_vote_std', 'size', 
                                                                    'CDR_sum']],age3_df[['pos_ex_value_sum','pos1_in_CDR3',
                                                                    'pos2_in_CDR3','pos3_in_CDR3']]]),index=age3_df.index), 
                                                                    age3_df.quality_imp, 
                                                                    test_size=0.2, stratify = age3_df.quality_imp)
X_train = pd.concat([X_flu_train, X_age3_train])
X_test = pd.concat([X_flu_test, X_age3_test])
y_train = pd.concat([y_flu_train, y_age3_train])
y_test = pd.concat([y_flu_test, y_age3_test])

clf.fit(X_train, y_train)

print "test roc_auc: ", roc_auc_score(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
print "age_test roc_auc: ", roc_auc_score(y_age3_test, clf.best_estimator_.predict_proba(X_age3_test)[:,1])
print "flu_test roc_auc: ", roc_auc_score(y_flu_test, clf.best_estimator_.predict_proba(X_flu_test)[:,1])

test roc_auc:  0.971540890698
age_test roc_auc:  0.934293643605
flu_test roc_auc:  0.975795356566


In [48]:
cls = RandomForestClassifier(n_estimators=100)
clf = GridSearchCV(estimator=cls, param_grid={'max_depth': range(1,2,10)}, scoring='roc_auc')

X_flu_train, X_flu_test, y_flu_train, y_flu_test = train_test_split(pd.DataFrame(scale(flu_df[['value1', 'size']]), index=flu_df.index), flu_df.quality_imp, 
                                                                    test_size=0.2, stratify = flu_df.quality_imp)
X_age3_train, X_age3_test, y_age3_train, y_age3_test = train_test_split(pd.DataFrame(scale(age3_df[['value1', 'size']]), index=age3_df.index), age3_df.quality_imp, 
                                                                    test_size=0.2, stratify = age3_df.quality_imp)
X_train = pd.concat([X_flu_train, X_age3_train])
X_test = pd.concat([X_flu_test, X_age3_test])
y_train = pd.concat([y_flu_train, y_age3_train])
y_test = pd.concat([y_flu_test, y_age3_test])

clf.fit(X_train, y_train)

print "age_df roc_auc: ", roc_auc_score(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])

age_df roc_auc:  0.966871150175


In [55]:
cls = RandomForestClassifier(n_estimators=3000)
clf = GridSearchCV(estimator=cls, param_grid={'max_depth': range(1,2,10)}, scoring='roc_auc')

X_flu_train, X_flu_test, y_flu_train, y_flu_test = train_test_split(pd.DataFrame(np.hstack([flu_df[['second_vote_std', 'size', 
                                                                    'CDR_sum','pos_ex_value_sum']],flu_df[['pos1_in_CDR3',
                                                                    'pos2_in_CDR3','pos3_in_CDR3']]]), index=flu_df.index), 
                                                                    flu_df.quality_imp, 
                                                                    test_size=0.2, stratify = flu_df.quality_imp)

X_age3_train, X_age3_test, y_age3_train, y_age3_test = train_test_split(pd.DataFrame(np.hstack([age3_df[['second_vote_std', 'size', 
                                                                    'CDR_sum', 'pos_ex_value_sum']],age3_df[['pos1_in_CDR3',
                                                                    'pos2_in_CDR3','pos3_in_CDR3']]]),index=age3_df.index), 
                                                                    age3_df.quality_imp, 
                                                                    test_size=0.2, stratify = age3_df.quality_imp)
X_train = pd.concat([X_flu_train, X_age3_train])
X_test = pd.concat([X_flu_test, X_age3_test])
y_train = pd.concat([y_flu_train, y_age3_train])
y_test = pd.concat([y_flu_test, y_age3_test])

clf.fit(X_train, y_train)

print "test roc_auc: ", roc_auc_score(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
print "age_test roc_auc: ", roc_auc_score(y_age3_test, clf.best_estimator_.predict_proba(X_age3_test)[:,1])
print "flu_test roc_auc: ", roc_auc_score(y_flu_test, clf.best_estimator_.predict_proba(X_flu_test)[:,1])

test roc_auc:  0.957433209659
age_test roc_auc:  0.919543533292
flu_test roc_auc:  0.967081260886


In [52]:
RandomForestClassifier?